First thing: check that this is running on the right computer:

In [1]:
import socket
assert socket.gethostname() == "lcs098116"

In [2]:
from pathlib import Path
base_path = Path('/media/storage/jim/speech-syn/Corpora/TG4')

Some imports for later:

In [3]:
from pydub import AudioSegment
import numpy as np
import datetime
import pandas as pd

In [4]:
df = pd.DataFrame(columns=["name", "seconds", "hms"])
df_cnt = 0

In [5]:
def sum_path_by_type(path, pattern):
    acc = []
    for audio in path.glob(pattern):
        audio = AudioSegment.from_file(str(audio))
        acc.append(audio.duration_seconds)
    return np.sum(acc)

In [ ]:
totals = {}
skipdirs = ["CloIarChonnacht", "no-subtitles-kids", "lm-data", "scripts", "unsorted-march"]
for subdir in base_path.glob('*'):
    dirname = subdir.stem
    if not subdir.is_dir() or dirname in skipdirs:
        continue
    totals[dirname] = sum_path_by_type(subdir, "*.wav")

In [ ]:
totals_hms = {a: str(datetime.timedelta(seconds=b)) for (a, b) in totals.items()}

In [ ]:
for dir in totals_hms.keys():
    print(f"{dir}: {totals[dir]} seconds ({totals_hms[dir]})")

In [ ]:
for scdir in totals_hms.keys():
    df.loc[df_cnt] = [f"{scdir}", totals[scdir], totals_hms[scdir]]
    df_cnt += 1

In [ ]:
df.to_csv("unaligned.csv")

In [ ]:
from IPython.display import display, HTML
display(HTML(df.to_html()))

In [ ]:
total_all = np.sum(df['seconds'])
total_all_hms = str(datetime.timedelta(seconds=total_all))
print(f'Total: {total_all} ({total_all_hms})')

In [ ]:
print(58*24)